# Load the pretrained model

Archive Images (on Server!)

In [ ]:
tar -czvf /home/alex/all_imgs_11_10_23.tar.gz

Download archive from server and extract it

In [ ]:
scp alex@foodsnapai.com:/home/alex/all_imgs_11_10_23.tar.gz allImgs.tar.gz
tar -xzvf allImgs.tar.gz -C /home/alex/allImgs_extracted/

Load annotations without Images

In [61]:
import os

# Get the value of the environment variable
labelstudio_token = os.getenv('LABELSTUDIO_TOKEN')
annotations_base_filename = "annotations_test3"
!curl -X GET http://foodsnapai.com:8080/api/projects/1/export?exportType=CSV -H 'Authorization: Token {labelstudio_token}' --output {annotations_base_filename +".csv"} 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2583k  100 2583k    0     0   342k      0  0:00:07  0:00:07 --:--:--  597k


Convert annotations and images and preprocess them

In [62]:
%run 'convert_csv_to_mlflow_csv.py' {annotations_base_filename +".csv"} 

In [63]:
# merge classes here
# curret merge string: not:Tomato,Cucumber,Bell-Pepper,Onion,Carrot,Banana,Egg,Scallion,Lemon,Potato,Zucchini,Garlic,Apple,Pumpkin,Ginger
%run 'convert_pascal_to_googlecsv.py' --merged_csv {annotations_base_filename +"_mlflow.csv"} 

Skipping automatic class merge.
['Tomato', 'Cucumber', 'Bell-Pepper', 'Onion', 'Carrot', 'Banana', 'Egg', 'Scallion', 'Lemon', 'Potato', 'Zucchini', 'Garlic', 'Apple', 'Pumpkin', 'Ginger']
Merging classes ['Soysauce', 'Peas', 'Oil', 'Tofu', 'Rice', 'Beans', 'Strawberry', 'Chilli', 'Lentils', 'Milk', 'Avocado', 'Spinach', 'Vinegar', 'Mango', 'Yoghurt', 'Canned-Tomato', 'Corn', 'Pasta', 'Chickpeas', 'Skyr', 'Balsamico', 'Nuts', 'Mushroom', 'Cheese', 'Plantmilk', 'Cauliflower', 'Flour', 'Broccoli', 'Orange', 'Butter', 'Curd', 'Cabagge', 'Lime', 'Eggplant', 'Salad', 'Leek'] to 'other'
Train-test-validation split applied to annotations_test3_mlflow_shuffled.csv


Now rename the file paths in the file to their actual location (find & replace)

In [64]:
import csv
import os

# Constant path to prepend
constant_path = '/home/alex/allImgs_extracted_smaller/'

# CSV file path
csv_file_path = annotations_base_filename +"_mlflow_shuffled.csv"

# Open the CSV file and modify the paths in the second column
with open(csv_file_path, 'r+', newline='') as file:
    csv_reader = csv.reader(file)
    rows = list(csv_reader)

    for row in rows:
        if len(row) > 1:  # Ensure the row has at least two columns
            # Assuming the file path is in the second column (index 1)
            file_path = row[1]
            file_name = os.path.basename(file_path)  # Extract the filename from the original path
            row[1] = os.path.join(constant_path, file_name)  # Create the new path with the constant path and filename

    # Move the file pointer to the beginning
    file.seek(0)

    # Write the modified rows back to the CSV file
    csv_writer = csv.writer(file)
    csv_writer.writerows(rows)

print(f"CSV file '{csv_file_path}' modified with constant path replacing each file path in the second column.")



CSV file 'annotations_test3_mlflow_shuffled.csv' modified with constant path replacing each file path in the second column.


In [65]:
!pwd
%run 'png_to_jpeg.py' {annotations_base_filename +"_mlflow_shuffled.csv"} 
#%run 'png_to_jpeg.py' annotations1650_mlflow_shuffled_n.csv

/home/alex/tflite_model_maker_wsl2
convert png image files to jpeg imgs


0it [00:00, ?it/s]

Converted 0
Ignored 11599 files because: Not a PNG file
Ignored 0 files because: Not found


In [66]:
# make images smaller (run only when new images have been loaded)

#%run preproc_imgs.py /home/alex/allImgs_extracted /home/alex/allImgs_extracted_smaller 800
#%run preproc_imgs.py /mnt/z/annotated_individual/images/ /home/alex/individual_tiny_extracted_smaller 800

In [67]:

%run normalize_csv.py {annotations_base_filename +"_mlflow_shuffled.csv"}  {annotations_base_filename +"_mlflow_shuffled_n.csv"} 

check the stats of the dataset splits

In [68]:
%run 'get_label_stats.py' {annotations_base_filename +"_mlflow_shuffled_n.csv"} 

There are 1709 different files in this set
There are 16 different classes in this set
Classes: other, Apple, Lemon, Tomato, Potato, Cucumber, Ginger, Banana, Egg, Garlic, Bell-Pepper, Onion, Scallion, Zucchini, Carrot, Pumpkin
other occurs in 1468 different files
Apple occurs in 201 different files
Lemon occurs in 244 different files
Tomato occurs in 458 different files
Potato occurs in 219 different files
Cucumber occurs in 428 different files
Ginger occurs in 163 different files
Banana occurs in 284 different files
Egg occurs in 254 different files
Garlic occurs in 213 different files
Bell-Pepper occurs in 379 different files
Onion occurs in 341 different files
Scallion occurs in 277 different files
Zucchini occurs in 208 different files
Carrot occurs in 304 different files
Pumpkin occurs in 119 different files

Class Distribution in Splits:

TRAIN:
other: 1177 files (26.27%)
Apple: 163 files (3.64%)
Lemon: 189 files (4.22%)
Tomato: 367 files (8.19%)
Potato: 172 files (3.84%)
Cucumbe

Now you can start to load the model and train!

In [75]:
import numpy as np
import os

from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

model_name = 'efficientdet-lite1' # EfficientDetLite1Spec must also be set accordingly!
custom_model_dir_name = 'model_1700'
epochs = 50
batch_size = 4
model_dir = 'models/'+model_name+'/'+custom_model_dir_name+'_e'+str(epochs)+'_b'+str(batch_size)
#spec = model_spec.get('efficientdet_lite1')
spec = object_detector.EfficientDetLite1Spec( # change this also to correct model spec
    model_name = model_name,
    model_dir='/home/alex/checkpoints_lite0_1700imgs/',
    hparams='grad_checkpoint=true,strategy=gpus',
    epochs=epochs, batch_size=batch_size,
    steps_per_execution=1, moving_average_decay=0,
    var_freeze_expr='(efficientnet|fpn_cells|resample_p6)',
    tflite_max_detections=25
)


# Load the dataset

In [76]:
import os
current_directory = os.getcwd()
print("Current Working Directory:", current_directory)
file_path = '/home/alex/tflite_model_maker_wsl2/'+annotations_base_filename +"_mlflow_shuffled_n.csv"

if os.path.exists(file_path):
    print(f"The file '{file_path}' exists.")
train_data, validation_data, test_data = object_detector.DataLoader.from_csv(file_path)


Current Working Directory: /home/alex/tflite_model_maker_wsl2
The file '/home/alex/tflite_model_maker_wsl2/annotations_test3_mlflow_shuffled_n.csv' exists.


# Train the model

In [77]:
model = object_detector.create(train_data, model_spec=spec, train_whole_model=True, validation_data=validation_data)

Epoch 1/50
341/341 [==============================] - 229s 616ms/step - det_loss: 1.4903 - cls_loss: 0.9690 - box_loss: 0.0104 - reg_l2_loss: 0.0732 - loss: 1.5635 - learning_rate: 0.0065 - gradient_norm: 1.9796 - val_det_loss: 1.5545 - val_cls_loss: 1.0088 - val_box_loss: 0.0109 - val_reg_l2_loss: 0.0732 - val_loss: 1.6277
Epoch 2/50
341/341 [==============================] - 209s 613ms/step - det_loss: 1.3158 - cls_loss: 0.8647 - box_loss: 0.0090 - reg_l2_loss: 0.0732 - loss: 1.3890 - learning_rate: 0.0050 - gradient_norm: 2.2537 - val_det_loss: 1.5071 - val_cls_loss: 0.9726 - val_box_loss: 0.0107 - val_reg_l2_loss: 0.0732 - val_loss: 1.5803
Epoch 3/50
341/341 [==============================] - 211s 618ms/step - det_loss: 1.2479 - cls_loss: 0.8221 - box_loss: 0.0085 - reg_l2_loss: 0.0732 - loss: 1.3211 - learning_rate: 0.0050 - gradient_norm: 2.3830 - val_det_loss: 1.4813 - val_cls_loss: 0.9657 - val_box_loss: 0.0103 - val_reg_l2_loss: 0.0732 - val_loss: 1.5545
Epoch 4/50
341/341 [==

### Write the model maker class list to a file!

In [79]:
import json
import os

if not os.path.exists(model_dir):
    os.makedirs(model_dir)

label_map = model.model_spec.config.label_map.as_dict()
# Writing the dictionary to a JSON file
with open(model_dir+'/label_map.json', 'w') as file:
    json.dump(label_map, file)

# run tensorboard to monitor

In [81]:
# !tensorboard --logdir=/home/alex/checkpoints_lite1_1700imgs/ --port=6006

Run on test data

In [82]:
model.evaluate(test_data)

43/43 [==============================] - 9s 158ms/step



{'AP': 0.19247203,
 'AP50': 0.28074333,
 'AP75': 0.21863717,
 'APs': 0.0,
 'APm': 0.10680747,
 'APl': 0.2255289,
 'ARmax1': 0.17313394,
 'ARmax10': 0.30513027,
 'ARmax100': 0.31492803,
 'ARs': 0.0,
 'ARm': 0.15844609,
 'ARl': 0.3594202,
 'AP_/other': 0.26882666,
 'AP_/Apple': 0.0428695,
 'AP_/Lemon': 0.16508158,
 'AP_/Tomato': 0.18646981,
 'AP_/Potato': 0.13956542,
 'AP_/Cucumber': 0.1766632,
 'AP_/Ginger': 0.20991604,
 'AP_/Banana': 0.26109135,
 'AP_/Egg': 0.39668977,
 'AP_/Garlic': 0.15039946,
 'AP_/Bell-Pepper': 0.20281202,
 'AP_/Onion': 0.20246477,
 'AP_/Scallion': 0.22812863,
 'AP_/Zucchini': 0.076759845,
 'AP_/Carrot': 0.17447785,
 'AP_/Pumpkin': 0.19733657}

### Export the model to tflite model

In [83]:
model.export(export_dir=model_dir)

2023-11-20 00:40:18.594223: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'resample_p7/PartitionedCall' has 1 outputs but the _output_shapes attribute specifies shapes for 3 outputs. Output shapes may be inaccurate.
2023-11-20 00:40:24.901211: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2023-11-20 00:40:24.901252: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2023-11-20 00:40:24.901439: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpbtutufjv
2023-11-20 00:40:24.954496: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2023-11-20 00:40:24.954527: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmpbtutufjv
2023-11-20 00:40:25.148389: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2023-11-20 00:40:26.309831: I tensorflow/cc/saved_

Estimated count of arithmetic ops: 3.640 G  ops, equivalently 1.820 G  MACs
Estimated count of arithmetic ops: 3.640 G  ops, equivalently 1.820 G  MACs


fully_quantize: 0, inference_type: 6, input_inference_type: 3, output_inference_type: 0
2023-11-20 00:42:43.881289: I tensorflow/compiler/mlir/lite/flatbuffer_export.cc:1963] Estimated count of arithmetic ops: 3.640 G  ops, equivalently 1.820 G  MACs



Run inference script for visual output

In [84]:
# needs to be after training for some reason

%run 'png_to_jpeg.py' {annotations_base_filename +"_mlflow_shuffled_n.csv"}


%run do_inference.py --input_csv {annotations_base_filename +"_mlflow_shuffled_n.csv"}\
                       --model_url {model_dir}/model.tflite --output_dir output_inference1 --label_file {model_dir}/label_map.json

convert png image files to jpeg imgs


0it [00:00, ?it/s]

Converted 0
Ignored 11599 files because: Not a PNG file
Ignored 0 files because: Not found
Predications will be saved to output_inference1


100%|██████████| 170/170 [07:53<00:00,  2.78s/it]
